# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
3.3155e18,"""../../../../data/SampleHDS.jso…","""13143845387153379054""","""8657453503103143397""","""3771212276336808813""","""13323365386677432644""",false
4.7281e18,"""../../../../data/SampleHDS.jso…","""6439611834997473006""","""17819957631771012379""","""3771212276336808813""","""5299272281861679337""",true
4.1146e18,"""../../../../data/SampleHDS.jso…","""14211726112529598900""","""5545919246893895043""","""11573635128553622714""","""13323365386677432644""",false
1.0327e18,"""../../../../data/SampleHDS.jso…","""6439611834997473006""","""10467201729357443397""","""3978160960232738741""","""13323365386677432644""",false
1.5340e19,"""../../../../data/SampleHDS.jso…","""13143845387153379054""","""1996989423146451549""","""3978160960232738741""","""10238911112168588107""",true
3.5712e18,"""../../../../data/SampleHDS.jso…","""14211726112529598900""","""15006263327417458201""","""14371057234109234193""",null,false
1.6286e19,"""../../../../data/SampleHDS.jso…","""14211726112529598900""","""13180447961118051598""","""3978160960232738741""","""10238911112168588107""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.4909e19,"""../../../../data/SampleHDS.jso…","""10091358896479347850""","""8238280312153044439""","""FirstMortgage30yr""","""3436631775462886029""","""Rejected"""
1.6201e19,"""../../../../data/SampleHDS.jso…","""9114135871143188226""","""9770049587156606231""","""FirstMortgage30yr""","""10347262449561983998""","""Accepted"""
3.3886e18,"""../../../../data/SampleHDS.jso…","""9307404785649530197""","""7932560218220485051""","""MoneyMarketSavingsAccount""","""3436631775462886029""","""Rejected"""
3.4581e18,"""../../../../data/SampleHDS.jso…","""9114135871143188226""","""5075655066139306349""","""BasicChecking""","""3436631775462886029""","""Rejected"""
1.2587e19,"""../../../../data/SampleHDS.jso…","""10091358896479347850""","""10764461451527727079""","""BasicChecking""","""12307256735095269243""","""Accepted"""
9.9317e16,"""../../../../data/SampleHDS.jso…","""9307404785649530197""","""14273482539750423445""","""UPlusFinPersonal""",null,"""Rejected"""
1.6928e19,"""../../../../data/SampleHDS.jso…","""9307404785649530197""","""1248891498331748327""","""BasicChecking""","""12307256735095269243""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.4129e19,"""../../../../data/SampleHDS.jso…","""294502122729887008""","""2477184327732879569""","""FirstMortgage30yr""","""4022772050193477096""","""Rejected"""
1.4005e19,"""../../../../data/SampleHDS.jso…","""16642277062984998385""","""1289496531846675497""","""FirstMortgage30yr""","""1197042907710687338""","""Accepted"""
2.7867e18,"""../../../../data/SampleHDS.jso…","""10656314053106298421""","""1702628380508896279""","""MoneyMarketSavingsAccount""","""4022772050193477096""","""Rejected"""
1.8860e18,"""../../../../data/SampleHDS.jso…","""16642277062984998385""","""4437993195886865598""","""BasicChecking""","""4022772050193477096""","""Rejected"""
3.1989e18,"""../../../../data/SampleHDS.jso…","""294502122729887008""","""6872958256664246808""","""BasicChecking""","""15977752003510543581""","""Accepted"""
1.5125e19,"""../../../../data/SampleHDS.jso…","""10656314053106298421""","""5614390113198976815""","""UPlusFinPersonal""",null,"""Rejected"""
7.8790e18,"""../../../../data/SampleHDS.jso…","""10656314053106298421""","""9064359960389778042""","""BasicChecking""","""15977752003510543581""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
filename=filename
Customer_MaritalStatus=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.3665e19,"""4019899605903276689""","""7276589196639585288""","""FirstMortgage30yr""","""691374055765711211""","""Rejected"""
3.3645e18,"""16541640453425203985""","""9255861037472946395""","""FirstMortgage30yr""","""12164639235766369226""","""Accepted"""
9.7685e18,"""5161233150239918876""","""14765304836763894909""","""MoneyMarketSavingsAccount""","""691374055765711211""","""Rejected"""
8.3691e18,"""16541640453425203985""","""2339627990839361510""","""BasicChecking""","""691374055765711211""","""Rejected"""
6.1651e18,"""4019899605903276689""","""13954393820199026525""","""BasicChecking""","""10554987349560122333""","""Accepted"""
1.6985e18,"""5161233150239918876""","""5990686133092312711""","""UPlusFinPersonal""",null,"""Rejected"""
5.4952e18,"""5161233150239918876""","""1009232282633260677""","""BasicChecking""","""10554987349560122333""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'